# LangChain Agent with Qiskit Docs MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-docs-mcp-server** via the Model Context Protocol (MCP).

The agent uses the documentation MCP server to:
- Retrieve SDK module lists and documentation pages
- Answer factual questions by quoting and citing Qiskit docs (RAG mode)
- Provide concise excerpts, example snippets, and resource URIs

## Architecture

```
┌─────────────┐     MCP Server       ┌──────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │      qiskit-docs-mcp-server       │
│    Agent    │                      │                                  │
└─────────────┘                      │  ┌────────────────────────────┐  │
                                     │  │  Qiskit Documentation      │  │
                                     │  │  SDK Module References     │  │
                                     │  └────────────────────────────┘  │
                                     │               │                  │
                                     └───────────────│──────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │   Qiskit Docs   │
                                            │   Repository    │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server with examples dependencies
pip install qiskit-docs-mcp-server[examples]

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your LLM provider API key for the agent.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [3]:
!pip install python-dotenv


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv(override=True)  # looks for .env in current working directory

print(os.getenv("OPENAI_API_KEY")[:8])
print(os.getenv("ANTHROPIC_API_KEY")[:8])
print(os.getenv("GOOGLE_API_KEY")[:8])

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"


# Verify configuration
print("Configuration status:")
print(f"  LLM Provider Key: {'✓ Set' if any(os.getenv(k) for k in ['OPENAI_API_KEY', 'ANTHROPIC_API_KEY', 'GOOGLE_API_KEY']) else '✗ Not set'}")

sk-proj-
sk-ant-a
AIzaSyAw
Configuration status:
  LLM Provider Key: ✓ Set


## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [5]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

Using OpenAI GPT-4o


In [ ]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
print("Using Anthropic Claude Sonnet")

In [2]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

Using Google Gemini Pro


In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [3]:
SYSTEM_PROMPT = """You are a knowledgeable Qiskit documentation assistant with access to the
qiskit-docs-mcp-server through the MCP server.

You can help users:
- Retrieve SDK module lists and documentation pages (list_sdk_modules, get_module_docs)
- Answer factual questions by quoting and citing Qiskit documentation
- Provide concise excerpts, example snippets, and resource URIs for follow-up

When answering:
- Prefer exact quotes or short excerpts from documentation and include the resource URI
- If multiple sources apply, summarize and list the sources used
- For code examples, indicate the documented origin and recommend how to run them
- If a topic is not found, explain the search steps and suggest related documentation pages
"""

## Create the MCP Client

This configures the connection to the qiskit-docs-mcp-server:

In [10]:
import sys
import os

def get_mcp_client():
    base_path = "/home/luke/Desktop/mcp-servers/qiskit-docs-mcp-server"
    venv_python = os.path.join(base_path, ".venv/bin/python")
    src_path = os.path.join(base_path, "src")

    return MultiServerMCPClient(
        connections={
            "qiskit-docs": {
                "transport": "stdio",
                "command": venv_python,
                "args": ["-m", "qiskit_docs_mcp_server"],
                "env": {
                    **os.environ,
                    "PYTHONPATH": src_path,
                    "PYTHONUNBUFFERED": "1"
                }
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the documentation data connection
- Makes tool calls much faster

In [11]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [12]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [13]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: List Qiskit SDK Modules

Let's ask the agent to retrieve the list of available SDK modules:

In [14]:
async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "What are the available SDK modules in Qiskit?")
    print(response)

  + Exception Group Traceback (most recent call last):
  |   File "/home/luke/Desktop/mcp-servers/qiskit-docs-mcp-server/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_46452/2034992582.py", line 1, in <module>
  |     async with mcp_client.session("qiskit-docs") as session:
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/luke/.local/share/uv/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/contextlib.py", line 210, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/luke/Desktop/mcp-servers/qiskit-docs-mcp-server/.venv/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 145, in session
  |     async with create_session(
  |                ^^^^^^^^^^^^^^^
  |   File "/home/luke/.local/share/uv/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/contex

### Example 2: Get Documentation About Quantum Circuits

Retrieve documentation on a specific topic:

In [8]:
async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Show me documentation about quantum circuits and how to create them")
    print(response)

  + Exception Group Traceback (most recent call last):
  |   File "/home/luke/Desktop/mcp-servers/qiskit-docs-mcp-server/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_46452/2602388470.py", line 1, in <module>
  |     async with mcp_client.session("qiskit-docs") as session:
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/luke/.local/share/uv/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/contextlib.py", line 210, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/luke/Desktop/mcp-servers/qiskit-docs-mcp-server/.venv/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 145, in session
  |     async with create_session(
  |                ^^^^^^^^^^^^^^^
  |   File "/home/luke/.local/share/uv/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/contex

### Example 3: Learn About VQE

Get documentation about Variational Quantum Eigensolver:

In [ ]:
async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "What is VQE and how do I implement it in Qiskit?")
    print(response)

### Example 4: Understand the Qiskit Transpiler

Get documentation on circuit transpilation:

In [ ]:
async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Explain the Qiskit transpiler and how it works")
    print(response)

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the documentation assistant:

In [ ]:
async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

## Custom Query Cell

Use this cell to ask the agent any documentation question:

In [ ]:
# Enter your question here:
MY_QUESTION = "What are the best practices for writing efficient Qiskit code?"

async with mcp_client.session("qiskit-docs") as session:
    agent = await create_agent_with_session(session)
    response = await ask_agent(agent, MY_QUESTION)
    print(response)

## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `list_sdk_modules` | Retrieve list of available Qiskit SDK modules |
| `get_module_docs` | Get detailed documentation for a specific module |
| `search_docs` | Search documentation by keywords and topics |
| `get_code_examples` | Retrieve code examples from documentation |

## Troubleshooting

### MCP server not found?
Make sure `qiskit-docs-mcp-server` is installed and in your Python path. Run:
```bash
pip install qiskit-docs-mcp-server[examples]
```

### Import errors?
Ensure all dependencies are installed:
```bash
pip install langchain langchain-mcp-adapters python-dotenv
```

### Connection issues?
- Verify the MCP server is running properly
- Check that the server command `qiskit-docs-mcp-server` is accessible from your shell
- Try running in a fresh terminal session